In [2]:
from scipy.io import loadmat

pytorch_result = loadmat("../pytorch_result.mat")

pytorch_result

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Thu Mar 16 13:34:40 2023',
 '__version__': '1.0',
 '__globals__': [],
 'gallery_f': array([[ 0.00556454,  0.04589299,  0.01724695, ...,  0.11134594,
         -0.03921181, -0.02453328],
        [ 0.03158579,  0.05014115, -0.01442602, ...,  0.11348172,
          0.0333238 , -0.03349413],
        [ 0.05423865,  0.08545972,  0.02670871, ...,  0.06510425,
          0.02819162, -0.00937978],
        ...,
        [ 0.09795539,  0.11336005, -0.09431969, ...,  0.03060464,
         -0.05639343, -0.03466449],
        [ 0.07561462,  0.09244677, -0.04668616, ...,  0.01470202,
         -0.02031903, -0.03196152],
        [ 0.07498889,  0.07028159, -0.0727989 , ..., -0.02903754,
         -0.0321567 , -0.00037526]], dtype=float32),
 'gallery_label': array([[ 1,  1,  1, ..., 18, 19, 19]]),
 'gallery_cam': array([[1, 1, 1, ..., 2, 1, 1]]),
 'query_f': array([[-0.02331403, -0.00470431, -0.00314464, ...,  0.1311092 ,
         -0.01616445, -0

In [4]:
import numpy as np

x = np.arange(10)
print(x)\

x1 = np.argwhere(x==x)
print(x1)

[0 1 2 3 4 5 6 7 8 9]
[[0]
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]
 [7]
 [8]
 [9]]
